In [2]:
import os
import shutil
from tqdm import tqdm
from wand.image import Image
from wand.color import Color
from wand.display import display

def deskew_image(inpath, outpath):
    with Image(filename=inpath) as img:
        img.deskew(0.4*img.quantum_range)
        img.save(filename=outpath)

def find_images(folder_path, extensions=['.jpg', '.png', '.jpeg', '.gif', '.bmp']):
    image_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if any(file.lower().endswith(ext) for ext in extensions):
                image_paths.append(os.path.join(root, file))
    return image_paths



In [3]:
folder_path = "data/sliced_cards/11-28-2023"
images = find_images(folder_path)

new_folder_path = "data/training_data"
os.makedirs(new_folder_path, exist_ok=True)

for i, image_path in enumerate(images):
    new_image_path = os.path.join(new_folder_path, f"{i:04d}.jpg")
    shutil.copy(image_path, new_image_path)

In [9]:
class_0_path = "data/training_data/empty"
class_1_path = "data/training_data/checked"

class_0_images = find_images(class_0_path)
class_1_images = find_images(class_1_path)

class_0_classifications = [0] * len(class_0_images)
class_1_classifications = [1] * len(class_1_images)

In [10]:
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Load and preprocess images
def load_images(image_paths):
    images = []
    for path in image_paths:
        img = Image.open(path).convert('L')  # Convert to grayscale
        img = img.resize((25, 25))  # Resize image
        img_array = np.array(img).flatten()  # Flatten image
        images.append(img_array)
    return images

# Paths to your images and corresponding labels
image_paths = class_0_images + class_1_images
labels = class_0_classifications + class_1_classifications

images = load_images(image_paths)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.3)

# Initialize classifier
classifier = SVC()

# Train classifier
classifier.fit(X_train, y_train)

# Evaluate classifier
accuracy = classifier.score(X_test, y_test)
print(f"Accuracy: {accuracy}")


Accuracy: 1.0


In [17]:
import pickle

# Save the trained model
with open('checkbox_model.pkl', 'wb') as f:
    pickle.dump(classifier, f)

# Load the trained model
with open('checkbox_model.pkl', 'rb') as f:
    loaded_classifier = pickle.load(f)


In [15]:
final_test_folder = "data/checkboxes001"
final_test_paths = find_images(final_test_folder)
final_test_images = load_images(final_test_paths)

classifier.predict(final_test_images)

os.makedirs('pred_0/', exist_ok=True)
os.makedirs('pred_1/', exist_ok=True)

predictions = classifier.predict(final_test_images)
for i, pred in enumerate(predictions):
    if pred == 0:
        shutil.copy(final_test_paths[i], 'pred_0/')
    else:
        shutil.copy(final_test_paths[i], 'pred_1/')
